In [23]:
from pathlib import Path
import sys
parent_dir = str(Path().resolve().parent)
sys.path.append(parent_dir)
import tqdm
import utils.aadt_predictor as ap
import numpy as np

In [16]:
HPMS_DIR = Path('../../data/processed_data/HPMS')
RESPONSE_VARS = ['AADT_MDV', 'AADT_HDV']
RF_PREDICTOR_VARS = ["STATEFP", "COUNTYFP", "F_SYSTEM", "THROUGH_LANES", "AADT"]
RANDOM_STATE = 42

In [17]:
predictor = ap.AADTPredictor(HPMS_DIR / 'hpms_aadt_subset.csv', None, random_state = RANDOM_STATE)

Loading data from ../../data/processed_data/HPMS/hpms_aadt_subset.csv


/Users/aviralchawla/Library/CloudStorage/OneDrive-UniversityofVermont/Work/TRC/HPMSTrafficData/scripts/utils/aadt_predictor.py:37: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data_full = pd.read_csv(self.data_path)


Full Data loaded successfully: 6598607 rows and 17 columns.
Pre-processing data...
ERROR: The data could not be subsetted. [None]
ERROR: The data could not be loaded. 'NoneType' object has no attribute 'shape'


In [40]:
out_all = []
for response_var in tqdm.tqdm(RESPONSE_VARS):
    predictor.response_var = response_var
    predictor.subset_train_data()
    predictor.initialize_model("Random Forest", random_state=RANDOM_STATE)
    predictor.split_data(RF_PREDICTOR_VARS, state_fips= None, test_size=1e-10)
    out = predictor.cross_validate_model(5)
    out_all.append(out)

  0%|          | 0/2 [00:00<?, ?it/s]

Training Data subsetted successfully with AADT_MDV: 4766630 rows and 17 columns.
Random Forest model initialized with- {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 1.0, 'random_state': 42}
Training and testing data split with test size 1e-10 on State None and not stratified ...


 50%|█████     | 1/2 [13:27<13:27, 807.70s/it]

Training Data subsetted successfully with AADT_HDV: 4766451 rows and 17 columns.
Random Forest model initialized with- {'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 1.0, 'random_state': 42}
Training and testing data split with test size 1e-10 on State None and not stratified ...


100%|██████████| 2/2 [25:07<00:00, 753.82s/it]


In [ ]:
for i, response_var in enumerate(RESPONSE_VARS):
    print(f"Response variable: {response_var}\n")
    print("--------------------------------------\n")
    print(f"R2 mean: {np.mean(out_all[i]['r2'])}, R2 std: {np.std(out_all[i]['r2'])}\n")
    print(f"RMSE mean: {np.mean(np.sqrt(out_all[i]['mse']))}, RMSE std: {np.std(np.sqrt(out_all[i]['mse']))}\n")
    print(f"MAE mean: {np.mean(out_all[i]['mae'])}, MAE std: {np.std(out_all[i]['mae'])}\n")
    print("--------------------------------------\n")

[{'r2': array([0.99363463, 0.99292024, 0.99268641, 0.99280783, 0.99284854]), 'mae': array([13.13158359, 13.20662944, 13.27853578, 13.24269601, 13.25749951]), 'mse': array([8016.79630897, 9015.45227094, 9372.12413165, 9197.38667888,
       9020.69569532])}, {'r2': array([0.9967703 , 0.99688174, 0.99663439, 0.99650894, 0.99666169]), 'mae': array([16.57363605, 16.20049106, 16.75112262, 16.91339954, 16.70918029]), 'mse': array([19219.04712729, 18357.55280339, 19787.80353451, 20545.22936315,
       19743.12598721])}]
